In [ ]:
import os
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from time import sleep
from pathvalidate import sanitize_filename

result_list=[]

for n in range(1, 672):
    url = "https://scourt.go.kr/portal/dcboard/DcNewsListAction.work?gubun=44&pageSize=5&pageIndex="+str(n)+"&seqnum=0&cbub_code=&searchOption=&searchWord=%C7%FC%BB%E7"
    raw = requests.get(url, headers={'User-Agent' : 'Mozilla/5.0'})
    html = BeautifulSoup(raw.text, 'html.parser')
    titles = html.select("tbody tr")
    
    for t in titles:
        
        onclick = t.select_one("a").attrs["onclick"]
        if len(onclick) == 39:
            seqnum = onclick[9:14]
            cbub_code = onclick[18:23]
        elif len(onclick) == 38:
            seqnum = onclick[9:13]
            cbub_code = onclick[17:22]
        elif len(onclick) == 37:
            seqnum = onclick[9:12]
            cbub_code = onclick[16:21]
            
        url_t = "https://scourt.go.kr/portal/dcboard/DcNewsViewAction.work?gubun=44&pageSize=5&pageIndex="+str(n)+"&seqnum="+seqnum+"&cbub_code="+cbub_code+"&searchOption=&searchWord="
        raw_t = requests.get(url_t, headers={'User-Agent' : 'Mozilla/5.0'})
        html_t = BeautifulSoup(raw_t.text, 'html.parser')
        
        raw_title = html_t.select_one("tr td").text
        
        title = sanitize_filename(raw_title)
        
        #경로 글자 제한.
        if len(title) > 240:
            title = title[-240:]
            
        file_len = len(html_t.select("td.attTxt a"))
        #한 게시글의 파일이 한 개인 경우만 다운로드.
        if file_len != 1:
            continue
            
        file_link = html_t.select_one("td.attTxt a").attrs['href']
        #파일의 형식에 맞게 저장.
        file_name = ''
        if 'pdf' in file_link:
            file_name = os.path.join(r'C:\pgm_file', '{}.pdf'.format(title))
        elif 'hwp' in file_link:
            file_name = os.path.join(r'C:\pgm_file', '{}.hwp'.format(title)) #[-8:]))
        elif 'HWP' in file_link:
            file_name = os.path.join(r'C:\pgm_file', '{}.HWP'.format(title)) #[-8:]))
        else:
            print(file_link)
            continue
            
        raw_file = requests.get(file_link, allow_redirects=True)
        
        with open(file_name, 'wb') as f:
            f.write(raw_file.content)
            
        new_dict={
            'seqnum': seqnum,
            'cbub_code': cbub_code,
            'title': raw_title,
            'url': url_t,
            'path': file_name
        }
        result_list.append(new_dict)
        
    sleep(2)
    
df = pd.DataFrame(result_list)        
txt_name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', f"{'xlsx_#1_file_list'}.xlsx")
excel_writer = pd.ExcelWriter(txt_name, engine='xlsxwriter')
df.to_excel(excel_writer)
excel_writer.save()